# Supertrend Long Trading Strategy with DeepAR Forecasting

In [ ]:
# Imports
import os
import json
import time
import boto3
import requests
import schedule
import numpy as np
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from botocore.config import Config
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### Adjust the pandas DataFrame width setting to avoid wrapping

In [ ]:
pd.set_option('display.width', 1200)
pd.set_option('display.max_columns', 20)

### Load the environment variables (.env is a hidden file)

In [ ]:
load_dotenv();

In [ ]:
# Set Alpaca API key, secret and url
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca_api_url = 'https://paper-api.alpaca.markets'

# Set historical data symbol, history, interval, frequency, and date window
symbol = "SPY"
history = 10
interval = 15
frequency = f"{interval}min"
timeframe = TimeFrame(interval, TimeFrameUnit.Minute)

# Set the range to filter data within normal market hours
market_open = pd.Timestamp("09:30:00", tz="America/Phoenix").time()
market_close = pd.Timestamp("16:00:00", tz="America/Phoenix").time()

# Create the Alpaca Trading API
alpaca = REST(
    alpaca_api_key, 
    alpaca_secret_key, 
    alpaca_api_url, 
    api_version = "v2"
)

### Functions to get the date window, historical data and DeepAR Forecast model inferences

In [ ]:
# Gets the current date window of days based on specified history
def get_date_window(history=10):
    timestamp = pd.Timestamp.now(tz="America/Phoenix").round(freq=frequency)
    start = (timestamp+pd.Timedelta(days=-history)).isoformat()
    end = timestamp.isoformat()
    return (start, end)

# Gets historical data from the Alpaca Trading API
def get_historical_data(symbol, timeframe, start, end, open=market_open, close=market_close, limit = 1000):
    historical_data = alpaca.get_barset(
        symbol,
        timeframe,
        start=start,
        end=end,
        limit=limit
    ).df[symbol]
    
    historical_data = historical_data.loc[(historical_data.index.time >= open) & (historical_data.index.time <= close)]
    historical_data.index = historical_data.index.tz_localize(None)
    return historical_data

In [ ]:
# Get the start and end of date window
start, end = get_date_window()

# Fetch the historical data for the symbol based on date window
historical_data = get_historical_data(symbol, timeframe, start, end)

# Verify that the date and times are within the market hours
historical_data

In [ ]:
# See how much the stock moved in the specified time window.
week_open = historical_data.iloc[0]["open"]
week_close = historical_data.iloc[-1]["close"]
percent_change = (week_close - week_open) / week_open * 100

# Display the price change percentage for the history range
print(f"{symbol} moved {percent_change:.2f}% over the last {history} days")

In [ ]:
# Set the AWS region and endpoint names, respectively
aws_region = os.getenv("AWS_REGION_NAME")
aws_endpoint = os.getenv("AWS_ENDPOINT_NAME")

# Create an AWS SDK configuration
aws_configuration = Config(
    region_name = aws_region,
    signature_version = "v4",
    retries = {
        "max_attempts": 5,
        "mode": "standard"
    }
)

# Create an instance of the boto3 client to access the SageMaker endpoint
aws = boto3.client('runtime.sagemaker', config=aws_configuration)

### Functions to get the DeepAR forecast model inferences

In [ ]:
# Build the DeepAR Forecasting request
def build_forecast_request(series, quantiles=["0.1", "0.5", "0.9"], encoding="utf-8"):
    instances = [{"start": str(data.index[0]), "target": list(data.close.values)} for group, data in series]
    configuration = { "output_types": ["mean", "quantiles"], "quantiles": quantiles }
    request = { "instances": instances, "configuration": configuration }    
    return json.dumps(request).encode(encoding)

# Get forecasted inferences from the AWS hosted DeepAR model
def get_forecast(request, endpoint=aws_endpoint, encoding="utf-8"):
    response  = aws.invoke_endpoint(
        ContentType="application/json",
        EndpointName=endpoint,
        Body=request
    )
    return json.loads(response["Body"].read().decode(encoding))["predictions"][-1]

In [ ]:
# True range function
def tr(data):
    data['prev-close'] = data['close'].shift(1) # Generate previous close
    data['high-low'] = abs(data['high'] - data['low']) # Candle high minus low
    data['high-pc'] = abs(data['high'] - data['prev-close']) # Candle high minus previous close
    data['low-pc'] = abs(data['low'] - data['prev-close']) # Candle low minus previous close
    return data[['high-low', 'high-pc', 'low-pc']].max(axis=1) # Find max of previous calculations

# Average true range function
def atr(data, period):
    data['true-range'] = tr(data)
    return data['true-range'].rolling(period).mean() # Get average of true range

# Supertrend function
def supertrend(df, period=10, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['avg-true-range'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['avg-true-range']) # Create lower ATR band
    df['lowerband'] = hl2 - (atr_multiplier * df['avg-true-range']) # Create upper ATR band
    df['in-uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]:
            df['in-uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in-uptrend'][current] = False
        else:
            df['in-uptrend'][current] = df['in-uptrend'][previous]

            if df['in-uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in-uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

# Global variables
in_position = False
current_bar_datetime = None
previous_bar_datetime = None

# Evaluate the forecasting against price and signal
def forecast_confirms_signal(order, historical_data, close_price):
    
    # Make the call to get the closing prices forecast
    series = historical_data.groupby(historical_data.index.date)
    request = build_forecast_request(series)
    forecast = get_forecast(request)
    
    # Print out the forecast values for reporting
    print(json.dumps(forecast, indent=4))
    
    # Calculate the forecasted mean close price
    forecast_price = round(np.mean(forecast["mean"]), 2)
    
    if order == "Buy":
        if forecast_price >= close_price:
            return True
        else: 
            print(f"*** {order} signal override. The close price of ${close_price:.2f} was greater than the forecasted average price of ${forecast_price:.2f} for the next 45 minutes. ***")
        return False
    elif order == "Sell":
        if forecast_price <= close_price:
            return True
        else: 
            print(f"*** {order} signal override. The close price of ${close_price:.2f} was less than the forecasted average price of ${forecast_price:.2f} for the next 45 minutes. ***")
            return False
    else:
        print(f"Order type '{order}' not recognized.  No action taken...")
        return True

# Order Execution Functions
def buy_market(symbol=symbol):
    alpaca.submit_order(
    symbol=symbol,
    qty=100,
    side='buy',
    type='market',
    time_in_force='gtc'
)
    
def sell_market(symbol=symbol):
    alpaca.submit_order(
    symbol=symbol,
    qty=100,
    side='sell',
    type='market',
    time_in_force='gtc'
)
    
# Check buy & sell signals function
def check_buy_sell_signals(df):
    global in_position

    print("Checking the supertrend indicator for buy/sell signals\n")
    print(df.tail(3))
    
    # Get the close price, current and previous uptrend flags
    close_price = df["close"][len(df.index) - 1]
    currently_in_uptrend = df["in-uptrend"][len(df.index) - 1]
    previously_in_uptrend = df["in-uptrend"][len(df.index) - 2]
    
    if not in_position:
        if previously_in_uptrend and currently_in_uptrend and forecast_confirms_signal("Buy", df, close_price):
            print("Established uptrend, Buy")
            order = alpaca.buy_market()
            print(json.dumps(order, indent=4))
            in_position = True
        
    if not previously_in_uptrend and currently_in_uptrend and forecast_confirms_signal("Buy", df, close_price):
        print("Changed to uptrend, Buy")
        if not in_position:
            order = alpaca.buy_market()
            print(json.dumps(order, indent=4))
            in_position = True
        else:
            print("You're already in a position, there is nothing to do...")
    
    if previously_in_uptrend and not currently_in_uptrend and forecast_confirms_signal("Sell", df, close_price):
        if in_position:
            print("Changed to downtrend, Sell")
            order = alpaca.sell_market()
            print(json.dumps(order, indent=4))
            in_position = False
        else:
            print("You're not in a position, there is nothing to sell...")

# Run bot function
def run_bot():
    global current_bar_datetime
    global previous_bar_datetime
    
    # Get the current date window, historical data and timestamp
    start, end = get_date_window()
    historical_data = get_historical_data(symbol, timeframe, start, end)
    timestamp = pd.Timestamp.now(tz="America/Phoenix").strftime('%m/%d/%Y at %I:%M:%S %p (%Z)')
    
    # Set the most current bar datetime (if any exists)
    if (len(historical_data.index) > 0):
        current_bar_datetime = historical_data.index[-1]

    # Only generate the supertrend and check buy/sell when a new bar is created
    if current_bar_datetime and (current_bar_datetime != previous_bar_datetime):
        print(f"\nRetrieving historical data on {timestamp}")                        
        
        # Generate Supertrend
        supertrend_data = supertrend(historical_data)

        # Check for buy/sell signal and execute order (if appropriate)
        check_buy_sell_signals(supertrend_data)

        # Update previous bar datetime
        previous_bar_datetime = current_bar_datetime

In [ ]:
schedule.every(1).minutes.do(run_bot)

while True:
    schedule.run_pending()
    time.sleep(1)